In [82]:
import tensorflow as tf
import tqdm
import datetime
import os 
import librosa
import numpy as np

import IPython.display as ipd

In [83]:
train_ds, test_ds = tf.keras.utils.audio_dataset_from_directory(
                            directory="./SpeechMovementCommandClassifier/speech_commands_v0.02",
                            batch_size=None,
                            validation_split=0.1,
                            seed=0,
                            output_sequence_length=16000,
                            subset='both'
                        )

Found 105835 files belonging to 36 classes.
Using 95252 files for training.
Using 10583 files for validation.


In [84]:
# alphanumeric order
labels = sorted(os.listdir("./SpeechMovementCommandClassifier/speech_commands_v0.02"))

idx_label_dict = {label: idx for idx, label in enumerate(labels)}
    
forward_label = idx_label_dict["forward"]
backward_label = idx_label_dict["backward"]
right_label = idx_label_dict["right"]
left_label = idx_label_dict["left"]

In [85]:
idel_wav_file_path = "./LESSI noise/Idel.wav"
idel_wav, sr = librosa.load(idel_wav_file_path)
idel_wav = librosa.resample(y=idel_wav, orig_sr=sr, target_sr=16000)

walk_wav_file_path = "./LESSI noise/Walk.wav"
walk_wav, sr = librosa.load(walk_wav_file_path)
walk_wav = walk_wav[50000:]
walk_wav = librosa.resample(y=walk_wav, orig_sr=sr, target_sr=16000)


def add_noise(speech_wav):
    r = tf.random.uniform(shape=(), minval=0, maxval=2, dtype=tf.int32)

    # idel
    if r == 0:
        pos = np.random.randint(low=0, high=len(idel_wav) - 16000)
        min_alpha = 0.2
        max_alpha = 0.4
        alpha = (max_alpha - min_alpha) * np.random.random() + min_alpha
        superimposed = alpha * speech_wav + (1 - alpha) * idel_wav[pos:pos+16000]
        return superimposed
    
    # walk
    else:
        pos = np.random.randint(low=0, high=len(walk_wav) - 16000)
        min_alpha = 0.4
        max_alpha = 0.5
        alpha = (max_alpha - min_alpha) * np.random.random() + min_alpha
        superimposed = alpha * speech_wav + (1 - alpha) * walk_wav[pos:pos+16000]
        return superimposed


def prepare_data(dataset):

    # Flatten: (num_samples, 1) -> (num_samples)
    dataset = dataset.map(lambda wav, label: (tf.reshape(wav, (-1,)), label))

    dataset = dataset.map(lambda wav, label:(add_noise(wav), label))

    return dataset

In [86]:
train_ds = train_ds.apply(prepare_data)

In [87]:
for speech_wav, label in train_ds.take(10):
    ipd.display(ipd.Audio(speech_wav, rate=16000))

2024-05-15 20:05:08.786481: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
